In [256]:
# Se importa módulo "FastAPI"
from fastapi import FastAPI
# Se importa pandas
import pandas as pd
# Se importa módulo "warnings" para que no aparezcan dichos mensaje
import warnings
warnings.filterwarnings("ignore")

# Se importa el csv "Movies" con sus correspondientes transformaciones
movies_funciones = pd.read_csv('Movies_Transformaciones.csv')




# Se eliminan valores nulos y se convierte a datetima
movies_funciones = movies_funciones.dropna(subset= ['release_date']).reset_index(drop = True)
movies_funciones['release_date'] = pd.to_datetime(movies_funciones['release_date'], format='%Y-%m-%d', errors='coerce')



In [257]:
# Se observan y reemplazan valors "-inf" y "Nan"
print(movies_funciones['return (porcent)'].isna().sum())
print(np.isinf(movies_funciones['return (porcent)']).sum())

# Reemplazo


34464
3505


In [258]:
len(movies_funciones)

45376

In [259]:
# Se cambian las opciones de visualización para poder observar el contenido completo de las columnas
pd.reset_option('display.max_colwidth') # En caso de volver a establecerlo sería "pd.reset_option('display.max_colwidth')" # pd.set_option('display.max_colwidth', None)

# Se amplia la visión para observar la totalidad de las columnas
pd.set_option('display.max_columns', None)

In [260]:
# Se instancia "FastAPI"
app = FastAPI()

# Función 1

In [261]:
# Creamos diccionario con los meses en español 
meses_a_numero = {'enero' : 1, 
                    'febrero' : 2,
                    'marzo' : 3,
                    'abril' : 4,
                    'mayo' : 5,
                    'junio' : 6,
                    'julio' : 7,
                    'agosto' : 8,
                    'septiembre' : 9,
                    'octubre' : 10,
                    'noviembre' : 11,
                    'diciembre': 12}

In [262]:
@app.get('/cantidad_filmaciones_mes/{Mes}')
def cantidad_filmaciones_mes(Mes):
    Mes = Mes.lower()
    
    mes_numero = meses_a_numero.get(Mes, None)
    if mes_numero is None:
        return {'Error' : 'Mes no válido'}
    
    
    peliculas_mes = movies_funciones[movies_funciones['release_date'].dt.month == mes_numero]
    
    return {'cantidad' : len(peliculas_mes)}


In [263]:
cantidad_filmaciones_mes('febrero')

{'cantidad': 3032}

In [264]:
agosto = movies_funciones[movies_funciones['release_date'].dt.month == 2]
len(agosto)

3032

# Función 2

In [265]:
def cantidad_filmaciones_dia(Dia):
    
    dias_semana_diccionario = {'lunes':0, 
                     'martes':1, 
                     'miercoles':2,
                     'miércoles':2, 
                     'jueves':3, 
                     'viernes':4, 
                     'sabado':5,
                     'sábado': 5, 
                     'domingo':6}
    Dia = Dia.lower()
    
    dias_semana = dias_semana_diccionario.get(Dia, None)   
    if dias_semana is None:
        return {'Error' : 'Día no válido'}
    
    
    peliculas_dia = movies_funciones[movies_funciones['release_date'].dt.dayofweek == dias_semana]
    
    return {'cantidad' : len(peliculas_dia)}

In [266]:
cantidad_filmaciones_dia('sábado')

{'cantidad': 5151}

# Función 3

In [267]:
# from unidecode import unidecode

def score_titulo(titulo):
    
    titulo = titulo.lower()
    
    peliculas_filtradas = movies_funciones.loc[movies_funciones['title'].str.lower() == titulo]
    
    
    if peliculas_filtradas.empty:
        return {'Error' : 'Titulo No Encontrado'}
    else: 
        resultados = []
        
        # Se realiza un bucle "for" ya que hay titulos que repiten su nombre
        for _, fila in peliculas_filtradas.iterrows():
            resultados.append({
                'Titulo': fila['title'],
                'Año de Estreno': fila['release_year'],
                'Popularidad': fila['popularity']
            })
        
        return resultados
        

In [268]:
score_titulo('Robin Hood')

[{'Titulo': 'Robin Hood', 'Año de Estreno': 1973, 'Popularidad': 11.179855},
 {'Titulo': 'Robin Hood', 'Año de Estreno': 1922, 'Popularidad': 0.611926},
 {'Titulo': 'Robin Hood', 'Año de Estreno': 2010, 'Popularidad': 10.56812},
 {'Titulo': 'Robin Hood', 'Año de Estreno': 1991, 'Popularidad': '5.683753'}]

# Función 4

In [269]:
#@app.get('/votos_titulos/{titulo_filmacion}')
def votos_titulo(titulo_filmacion):
    
    titulo_filmacion = titulo_filmacion.lower() # El valor de entrada se pasa a minúscula para mejor manejo
    
    peliculas_filtradas = movies_funciones.loc[movies_funciones['title'].str.lower() == titulo_filmacion] # Se realiza el filtro de el título ingresado por el usuario con los titulos del DataFrame

    
    # Se utilizan condicionales
    if peliculas_filtradas.empty: # Si "peliculas_filtras" se encuentra vacío, devuelve "Título No Encontrado"
        return {'Error' : 'Titulo No Encontrado'}
    else: 
        peliculas_cantidad_votos = peliculas_filtradas[peliculas_filtradas['vote_count'] >= 2000] # Se realiza un filtro para dar con las películas que cuentan con más de 2000 valoraciones
        if peliculas_cantidad_votos.empty: # Si la variable "peliculas_cantidad_votos" se encuentra encuentra vacía, tiene menos de 2000 valoraciones y devuelve mensaje de error 
            return {'Error' : 'La película cuenta con menos de 2000 valoraciones. Ingrese un título que cumpla dicha condición'}
        else: # En el caso de cumplir con el filtro devuelve "Título", "Año Estreno (con motivo de especificar que hay titulos repetidos pero son distintas películas)", "Cantidad Votos", "Valor Promedio Votaciones"
        
            resultados = []
            
            for _, fila in peliculas_cantidad_votos.iterrows(): # Se realiza un bucle for sobre la variable "peliculas_cantidad_votos" ya que es donde se aplicó el filtro
                resultados.append({
                    'Titulo': fila['title'],
                    'Año Estreno' : fila['release_year'],
                    'Cantidad Votos': fila['vote_count'],
                    'Valor Promedio de Votaciones': fila['vote_average']
                })
                        
            return resultados
                    


In [270]:
print(votos_titulo('titanic'))

[{'Titulo': 'Titanic', 'Año Estreno': 1997, 'Cantidad Votos': 7770.0, 'Valor Promedio de Votaciones': 7.5}]


# Función 5

In [271]:
# Ya que las siguientes 2 funciones requieren datos del csv "credits", se procede a importar el mismo
credits_funciones = pd.read_csv('Credits_Transformaciones.csv')

In [272]:
import numpy as np

# Se observan y reemplazan valors "-inf" y "Nan"
print(movies_funciones['return (porcent)'].isna().sum())
print(np.isinf(movies_funciones['return (porcent)']).sum())

# Se reemplazan valores 
movies_funciones['return (porcent)'].fillna(0, inplace = True)
movies_funciones['return (porcent)'].replace(-np.inf, 0, inplace = True)

# Se observan nuevamente valores nulos y "-inf"
print(movies_funciones['return (porcent)'].isna().sum())
print(np.isinf(movies_funciones['return (porcent)']).sum())

34464
3505
0
0


In [273]:
def get_actor(nombre_actor): 
    
    # Se establece el mismo tipo de dato para las columnas "id_ORIGINAL" de ambos dataframe
    credits_funciones['id_ORIGINAL'] = credits_funciones['id_ORIGINAL'].astype(str)
    movies_funciones['id_ORIGINAL'] = movies_funciones['id_ORIGINAL'].astype(str)
    
    # El valor de entrada se pasa a minúscula para mejor manejo
    nombre_actor = nombre_actor.lower()
    
    # Se realiza el filtro de el título ingresado por el usuario con los titulos del DataFrame
    actor_filtrado = credits_funciones.loc[credits_funciones['Actores'].str.lower().str.contains(nombre_actor)] # Se realiza el filtro de el título ingresado por el usuario con los titulos del DataFrame

    
    if actor_filtrado.empty: # Si "actor_filtrado" se encuentra vacío, devuelve "Actor No Encontrado"
        return {'Error' : 'Actor No Encontrado'}
    else: # Caso contrario:
        
        # Une ambos dataframe por la columna "id_ORIGINAL"
        peliculas_actor = pd.merge(actor_filtrado, movies_funciones, on = 'id_ORIGINAL')
        
        # Calcula la cantidad total de películas en las que trabajó el actor
        cantidad_peliculas = len(peliculas_actor)
        
        # Calcula el promedio del retorno de las películas 
        promedio_retorno = (peliculas_actor['return (porcent)'].sum()) / cantidad_peliculas
        
        # Calcula el éxito del actor sumando el retorno de las películas
        exito_actor = peliculas_actor['return (porcent)'].sum()
        
        
        # Retorna el diccionario con dichos datos
        return {'Actor' : nombre_actor, 'Cantidad Peliculas' : cantidad_peliculas, 'Éxito Actor' : exito_actor, 'Promedio Retorno de Películas' : promedio_retorno}

In [274]:
get_actor('Jim Carrey')

{'Actor': 'jim carrey',
 'Cantidad Peliculas': 42,
 'Éxito Actor': 1290.2361894224387,
 'Promedio Retorno de Películas': 30.719909271962827}

# Función 6

In [275]:
def get_director(nombre_director):
    
    # Se establece el mismo tipo de dato para las columnas "id_ORIGINAL" de ambos dataframe
    credits_funciones['id_ORIGINAL'] = credits_funciones['id_ORIGINAL'].astype(str)
    movies_funciones['id_ORIGINAL'] = movies_funciones['id_ORIGINAL'].astype(str)
    
    
    # El valor de entrada se pasa a minúscula para mejor manejo
    nombre_director = nombre_director.lower()
    
    # Se realiza el filtro de el director ingresado por el usuario con los directores del DataFrame
    director_filtrado = credits_funciones.loc[credits_funciones['Director'].str.lower().str.contains(nombre_director)] 

    if director_filtrado.empty: # Si "director_filtrado" se encuentra vacío, devuelve Director No Encontrado"
        return {'Error' : 'Director No Encontrado'}
    
    else: # Caso contrario:
                
        # Une ambos dataframe por la columna "id_ORIGINAL"
        peliculas_director = pd.merge(director_filtrado, movies_funciones, on = 'id_ORIGINAL')
        
        # Calcula el éxito del director sumando el retorno de las películas
        exito_director = peliculas_director['return (porcent)'].sum()

        # Se extraen los títulos de las películas
        nombre_peliculas = peliculas_director['title'].tolist() # El método ".tolist()" convierte los valores en listas
        
        # Se extrae la fecha de lanzamiento
        fecha_lanzamiento = peliculas_director['release_date'].astype(str).tolist()
        
        # Se extrae el retorno individual 
        retorno_individual = peliculas_director['return (porcent)'].tolist()
        
        # Se extrae el costo
        costo = peliculas_director['budget'].tolist()
        
        # Se extrae la ganancia
        ganancia = peliculas_director['revenue'].tolist()
        
        
        # Se crea una lista donde se guardarán la información de cada película en formato "diccionario"
        peliculas_info = []
    for i in range(len(nombre_peliculas)): # El bucle "for" itera desde "0" hasta el útlimo índice de "nombre_peliculas"
        peliculas_info.append({ 
            'Título': nombre_peliculas[i],
            'Fecha Lanzamiento': fecha_lanzamiento[i],
            'Retorno Individual': retorno_individual[i],
            'Costo': costo[i],
            'Ganancia': ganancia[i]})
    
    # Retorna los valores detallados 
    return {'Director': nombre_director,
        'Éxito Director': exito_director,
        'Películas': peliculas_info}
        
         
    
    
    
    

In [276]:
get_director('john lasseter')

{'Director': 'john lasseter',
 'Éxito Director': 379.1407694867655,
 'Películas': [{'Título': 'Toy Story',
   'Fecha Lanzamiento': '1995-10-30',
   'Retorno Individual': 91.96903329912652,
   'Costo': 30000000,
   'Ganancia': 373554033},
  {'Título': "A Bug's Life",
   'Fecha Lanzamiento': '1998-11-25',
   'Retorno Individual': 66.96570585220056,
   'Costo': 120000000,
   'Ganancia': 363258859},
  {'Título': 'Toy Story 2',
   'Fecha Lanzamiento': '1999-10-30',
   'Retorno Individual': 81.90470543786864,
   'Costo': 90000000,
   'Ganancia': 497366869},
  {'Título': 'Luxo Jr.',
   'Fecha Lanzamiento': '1986-08-17',
   'Retorno Individual': 0.0,
   'Costo': 0,
   'Ganancia': 0},
  {'Título': 'Cars',
   'Fecha Lanzamiento': '2006-06-08',
   'Retorno Individual': 74.02502661412007,
   'Costo': 120000000,
   'Ganancia': 461983149},
  {'Título': 'Cars 2',
   'Fecha Lanzamiento': '2011-06-11',
   'Retorno Individual': 64.27629828344969,
   'Costo': 200000000,
   'Ganancia': 559852396},
  {'Tít